In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import asyncio
import random
import numpy as np
import time
import threading
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
from tqdm import tqdm
import re
import json
import aiohttp


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#book_ids = np.arange(1,21, 1)

book_id = []
for i in range(1000):
    m = random.choice(np.arange(1, 8987774, 1))
    book_id.append(m)
book_id

urls = []
for ID in tqdm(book_id):
    url = "https://www.goodreads.com/book/show/" + str(ID)
    urls.append(url)
    
urls

In [4]:
#book_id = np.arange(1, 8987774, 1)
book_id = np.arange(1, 2501, 1)
urls = []
for ID in tqdm(book_id):
    url = "https://www.goodreads.com/book/show/" + str(ID)
    urls.append(url)

100%|██████████| 2500/2500 [00:00<00:00, 344790.21it/s]


In [5]:
all_data = []
master_dict = {}
page_html = {}
titles = []
descriptions = []
genres = []
authors = []
contributors = []
ratings = []
pages=[]
raters = []
isbn13s =[]
isbn10s=[]
languages =[]
formats=[]
publishers =[]
image_links = []
amazon_links = []
book_ids = []
book_types = []
series_part_numbers =[]
series_names = []
reviews = []
awards = []
dates = []

async def extract_data(text):
    page_html = BeautifulSoup(text, 'html.parser')
    
    title= page_html.find("h1", {"class": "Text Text__title1", "data-testid":"bookTitle"})
    genre = []
    for a in page_html.select('a[href*="/genres/"]'):
        genre.append(a.text)
        
    
    contributor = []
    for b in page_html.find_all("span", {"class": "ContributorLink__name", "data-testid":"name"}):
        contributor.append(b.text)
    contributor = list(set(contributor))


    if page_html.find("span", {"class": "ContributorLink__name", "data-testid":"name"}) is not None:
        author = page_html.find("span", {"class": "ContributorLink__name", "data-testid":"name"}).text
    else:
        author = None

    if page_html.find("span", {"class": "Formatted"}) is not None:
        description = page_html.find("span", {"class": "Formatted"}).text
    else:
        description = None
    
    if page_html.find("div", {"class": "RatingStatistics__rating"}) is not None:
        rating = page_html.find("div", {"class": "RatingStatistics__rating"}).text
    else:
        rating = None
    
    if page_html.find("span", {"data-testid": "ratingsCount"}) is not None:
        rater = page_html.find("span", {"data-testid": "ratingsCount"}).text.split()[0]
    else:
        rater = None
    
    if page_html.find("span", {"data-testid": "reviewsCount"}) is not None:
        review = page_html.find("span", {"data-testid": "reviewsCount"}).text.split()[0]
    else:
        review = None
    

        
    if page_html.find("span", {"class": "Formatted"}) is not None:
        descTag = page_html.find("div", {"class": "BookPageMetadataSection__description", "data-testid":"description"})

        for tag in descTag:
            description = tag.find("span", {"class": "Formatted"}).text

    else:
        description = None
        
        
    if page_html.find("h3", {"class": "Text Text__title3 Text__italic Text__regular Text__subdued"}) is not None:

        series_info = page_html.find("h3", {"class": "Text Text__title3 Text__italic Text__regular Text__subdued"}).text

        if "#" in series_info:
            series_part_number = page_html.find("h3", {"class": "Text Text__title3 Text__italic Text__regular Text__subdued"}).text.split()[-1][1:]
            series_name= page_html.find("h3", {"class": "Text Text__title3 Text__italic Text__regular Text__subdued"}).text.split()[:-1]
            series_name = " ".join(series_name)
        else:
            series_part_number = None
            series_name= page_html.find("h3", {"class": "Text Text__title3 Text__italic Text__regular Text__subdued"}).text  
    else:
        series_part_number = None
        series_name = None
        
    original_title= page_html.find_all("script")
    i = json.loads(original_title[6].text)
    j = json.loads(original_title[5].text)


    for g in range(len(list(i['props']['pageProps']['apolloState'].keys()))):
        if list(i['props']['pageProps']['apolloState'].keys())[g][:10] == 'Book:kca:/':
            h = list(i['props']['pageProps']['apolloState'].keys())[g]
        else:
            continue


    try:
        isbn10 = i['props']['pageProps']['apolloState'][h]['details']['isbn']
    except:
        isbn10 = None
        
    try:
        date= i['props']['pageProps']['apolloState'][h]['details']['publicationTime']

    except:
        date = None

    try:
        isbn13 = i['props']['pageProps']['apolloState'][h]['details']['isbn13']
    except:
        isbn13 = None

    try:
        language = j['inLanguage']

    except:
        language = None


    try:
        page= j['numberOfPages']

    except:
        page = None

    try:
        formata = j['bookFormat']


    except:
        formata = None


    try:
        publisher = i['props']['pageProps']['apolloState'][h]['details']['publisher']

    except:
        publisher = None

    try:
        image_link = j['image']

    except:
        image_link = None


    try:
        award = j['awards']

    except:
        award = None

    try:
        book_type = i['props']['pageProps']['apolloState'][h]['bookSeries'][0]['__typename']

    except:
        book_type = 'Standalone'        
    


    return title.text, description, genre, author, contributor, rating, rater, review, series_part_number,series_name, isbn10, date, isbn13, language, page, formata, publisher, image_link, award, book_type


async def fetch(session, url):
    try:
        async with session.get(url) as response:
            text = await response.text()
            extract = await extract_data(text)
            titles = extract[0]
            descriptions = extract[1]
            genres = extract[2]
            authors= extract[3]
            contributors = extract[4]
            ratings = extract[5]
            raters = extract[6]
            reviews = extract[7]
            series_part_numbers = extract[8]
            series_names = extract[9]
            isbn10s = extract[10]
            dates = extract[11]
            isbn13s = extract[12]
            languages = extract[13]
            pages = extract[14]
            formatas = extract[15]
            publishers = extract[16]
            image_links = extract[17]
            awards = extract[18]
            book_types = extract[19]
            
            return url, titles, descriptions, genres, authors, contributors, ratings, raters, reviews, series_part_numbers,series_names, isbn10s, dates, isbn13s, languages, pages, formatas, publishers, image_links, awards, book_types

    except Exception as e:
        print(str(e))

async def main():
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in urls:
            tasks.append(fetch(session, url))

        htmls = await asyncio.gather(*tasks)
        all_data.extend(htmls)

        # Storing the raw HTML data.
        for html in tqdm(htmls):
            if html is not None:
                url = html[0]
                book_id = url[36:]
                master_dict[url] = {'Book ID':book_id, 'Book Title': html[1], 'Description': html[2],
                                    'Genres': html[3], 'Author': html[4], 'Contributors': html[5], 'Rating': html[6],
                                   'Number of Raters': html[7], 'Reviews Count': html[8], 'Series Part Number': html[9], 'Series Name': html[10],
                                   'ISBN-10': html[11],'Publication Date': html[12],'ISBN-13': html[13],
                                    'Language': html[14],'Number of Pages': html[15],'Format': html[16],
                                    'Publisher': html[17], 'Image Links': html[18], 'Awards': html[19],
                                    'Book Type': html[20],                     
}
            else:
                continue

await main()

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Cannot connect to host www.goodreads.com:443 ssl:default [None]
Cannot connect to host www.goodreads.com:443 ssl:default [None]
Cannot connect to host www.goodr

100%|██████████| 2500/2500 [00:00<00:00, 190882.71it/s]


In [6]:
#list(master_dict.values())

In [7]:
#all_data

In [8]:
new_cols = ['Book ID', 'Book Title', 'Description', 'Genres',
       'Author', 'Contributors', 'Rating', 'Number of Raters', 'Reviews Count', 'Book Type',
       'Series Part Number', 'Series Name', 'Publication Date','ISBN-10',
       'ISBN-13', 'Language', 'Number of Pages', 'Format', 'Publisher','Awards',
       'Image Links', 'Goodreads Page']

In [9]:
final_df = pd.DataFrame(master_dict).T.reset_index()
final_df.rename(columns = {'index': 'Goodreads Page'}, inplace =True)
final_df = final_df.reindex(columns = new_cols)
final_df

,Book ID,Book Title,Description,Genres,Author,Contributors,Rating,Number of Raters,Reviews Count,Book Type,...,Publication Date,ISBN-10,ISBN-13,Language,Number of Pages,Format,Publisher,Awards,Image Links,Goodreads Page
0,1,Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...","[Fantasy, Young Adult, Fiction, Magic, Childre...",J.K. Rowling,[J.K. Rowling],4.58,"3,053,535","52,203",BookSeries,...,1158390000000,None,None,English,652,Paperback,Scholastic Inc.,"Locus Award Best Young Adult Novel (2006), Aud...",https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/1
1,2,Harry Potter and the Order of the Phoenix,Harry Potter is about to start his fifth year ...,"[Fantasy, Young Adult, Fiction, Magic, Childre...",J.K. Rowling,"[J.K. Rowling, Mary GrandPré]",4.50,"3,153,138","57,286",BookSeries,...,1094022000000,None,None,English,912,Paperback,Scholastic Inc.,Bram Stoker Award Works for Young Readers (200...,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/2
2,3,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...,"[Fantasy, Fiction, Young Adult, Magic, Childre...",J.K. Rowling,[J.K. Rowling],4.47,"9,294,866","147,588",BookSeries,...,1067673600000,None,None,English,309,Hardcover,Scholastic Inc,Mythopoeic Fantasy Award Children&apos;s Liter...,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/3
3,4,Harry Potter and the Chamber of Secrets,The Dursleys were so mean and hideous that sum...,"[Fantasy, Fiction, Young Adult, Magic, Childre...",J.K. Rowling,[J.K. Rowling],4.43,"3,605,003","71,584",BookSeries,...,1067673600000,None,None,English,352,Hardcover,Scholastic,Mythopoeic Fantasy Award Children&apos;s Liter...,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/4
4,5,Harry Potter and the Prisoner of Azkaban,"Harry Potter, along with his best friends, Ron...","[Fantasy, Fiction, Young Adult, Magic, Childre...",J.K. Rowling,"[J.K. Rowling, Mary GrandPré]",4.58,"3,816,633","75,387",BookSeries,...,1083394800000,043965548X,9780439655484,English,435,Mass Market Paperback,Scholastic Inc.,Bram Stoker Award Best Work for Young Readers ...,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,1610,Political Philosophy and Rhetoric: A Study of ...,This book analyses the origins of modern party...,[],John Zvesper,[John Zvesper],3.00,2,0,Standalone,...,231058800000,0521213231,9780521213233,English,224,Hardcover,Cambridge University Press,None,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/1610
1420,1611,The Structure of Ancient Arguments: Rhetoric a...,"NY 1986 1st Shapolsky/Steinmatzky. 8vo., 138pp...",[],Ronald C. Katz,[Ronald C. Katz],4.00,4,1,Standalone,...,504950400000,0933503342,9780933503342,English,138,Hardcover,Sure Sellers Inc,None,,https://www.goodreads.com/book/show/1611
1421,1612,Defining Reality: Definitions and the Politics...,"In Defining Reality, Edward Schiappa argues th...",[Philosophy],Edward Schiappa,[Edward Schiappa],4.39,18,2,BookSeries,...,1052118000000,0809325012,9780809325016,English,232,Paperback,Southern Illinois University Press,None,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/1612
1422,1613,Protagoras and Logos: A Study in Greek Philoso...,Reassesses the philosophical and pedagogical c...,[Philosophy],Edward Schiappa,[Edward Schiappa],4.12,17,1,BookSeries,...,1064991600000,1570035210,9781570035210,English,272,Paperback,University of South Carolina Press,None,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/1613


In [10]:
len(final_df)

1424

In [11]:
final_df.to_csv("Goodreads Books Data.csv", index=False)